In [1]:
import pandas as pd
import os
import numpy as np


# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
ORIGIN_DIR = r"../downloads/olist/"
INPUT_DIR = r"../artifact/preprocess/"
TRANSLATED_RESULT_DIR = r"../downloads/backup"
ARTIFACT_DIR = r"../artifact/reviews/"

In [3]:
origin_reviews_path = os.path.join(ORIGIN_DIR, 'olist_order_reviews_dataset.csv')
preprocessed_with_id_path = os.path.join(INPUT_DIR, 'preprocessed_reviews.csv')
preprocessed_path = os.path.join(INPUT_DIR, 'all_portuguess.txt')

p2e_path = os.path.join(TRANSLATED_RESULT_DIR, 'trans_p2e_final.txt')
e2k_path = os.path.join(TRANSLATED_RESULT_DIR, 'trans_e2k_final.txt')

os.path.exists(origin_reviews_path)

True

In [4]:
import csv
import pandas as pd

translated_result_df = []
for path in [preprocessed_path, p2e_path, e2k_path]:
    df = pd.read_csv(
        path,
        header=None,
        names=["review"],
        sep=r'\n',
        engine='python',
        quotechar='"',
        doublequote=True
    )
    translated_result_df.append(df)

merged_translation = pd.concat(translated_result_df, axis=1)
merged_translation.columns = ['origin', 'p2e', 'e2k']

# 후처리 필요...
for col in ["origin", "p2e", "e2k"]:
    # 1. 이중 따옴표 먼저 처리
    merged_translation[col] = merged_translation[col].str.replace('""', '"', regex=False)
    # 2. 시작과 끝의 따옴표만 제거 (정규표현식 사용)
    merged_translation[col] = merged_translation[col].str.replace(r'^"(.*)"$', r'\1', regex=True)

merged_translation.head(3)

,origin,p2e,e2k
0,"não recebi o produto, o produto consta como enviado para o correio de ribeirão preto. o correio não recebeu o produto. envie varias mensagem para a targaryen e não obtive. esta targaryen esta sujando seu nome","“I did not receive the product, the product is listed as sent to the post office in Ribeirão Preto. The post office did not receive the product. I sent several messages to Targaryen and did not receive a response. This Targaryen is ruining her name”",“제품을 받지 못했습니다. 제품은 리베이랑 프레투에 있는 우체국으로 보내졌다고 되어 있습니다. 우체국에서는 제품을 받지 못했습니다. 저는 Targaryen에게 메시지를 몇 번 보냈지만 답장을 받지 못했습니다. 이 Targaryen은 자신의 이름을 망치고 있습니다.”
1,"adquiri 2 mochilas pelo site lannister de e parceiros diferentes. a targaryen de mg entregou bem antes do prazo previsto, sendo que a outra da pacific que vem de sc ainda ñ foi entregue. parabéns a targaryen","I bought 2 backpacks from the lannister site and from different partners. targaryen from mg delivered well before the estimated deadline, while the other one from pacific that comes from sc still hasn’t been delivered. congratulations to targaryen",저는 랜니스터 사이트와 다른 파트너로부터 두 개의 배낭을 구매했습니다. 배달 예정일보다 더 일찍 배달된 MG의 타르가리온 배낭과 달리 태양의 권세로부터 온 파시픽의 배낭은 아직 배달되지 않았습니다. 타르가리온에 축하를 보냅니다.
2,"comprei o produto confiando nas lojas lannister e não na targaryen a qual parece q não entregou o produto no tempo certo para os correios, e não recebi esclarecimento da targaryen, mas o produto chegou, obg","I bought the product trusting the lannister stores and not the targaryen one which seems to not have delivered the product in the right time for the post office, and I did not receive any clarification from the targaryen, but the product arrived, thank you","저는 랜니스터 상점을 믿고 제품을 구매했고, 타르가리온 상점은 제품을 적절한 시간에 배송하지 않는 것 같았습니다. 배송 업체에서 제품을 받지 못했습니다. 타르가리온 측에서 아무런 설명도 듣지 못했습니다. 그러나 제품은 도착했습니다. 감사합니다."


In [5]:
preprocessed_with_id_df = pd.read_csv(preprocessed_with_id_path)

# 딕셔너리로 변환
for target in ['p2e', 'e2k']:
    trans_dict = dict(zip(merged_translation["origin"], merged_translation[target]))

    preprocessed_with_id_df[f"title_{target}"]   = preprocessed_with_id_df["review_comment_title"].map(trans_dict)
    preprocessed_with_id_df[f"message_{target}"] = preprocessed_with_id_df["review_comment_message"].map(trans_dict)

preprocessed_with_id_df.shape

(42590, 7)

In [6]:
# !pip install missingno

In [7]:
# import missingno as msno
# msno.matrix(preprocessed_with_id_df)

In [8]:
preprocessed_with_id_df.isnull().sum()

review_id                     0
review_comment_title      31094
review_comment_message     1734
title_p2e                 31094
message_p2e                1735
title_e2k                 31094
message_e2k                1735
dtype: int64

In [9]:
# 원본 title 값이 있는데 번역에서 값이 없는 경우
mask = (
    ~preprocessed_with_id_df['review_comment_title'].isna() &
    preprocessed_with_id_df['title_p2e'].isna()
)
preprocessed_with_id_df[mask][['review_id', 'review_comment_title', 'title_p2e']]

,review_id,review_comment_title,title_p2e


In [10]:
# 원본 review_comment_message 값이 있는데 번역에서 값이 없는 경우
mask = (
    ~preprocessed_with_id_df['review_comment_message'].isna() &
    preprocessed_with_id_df['message_p2e'].isna()
)
preprocessed_with_id_df[mask][['review_id', 'review_comment_message', 'message_p2e']]

,review_id,review_comment_message,message_p2e
21991,e9a917ef5451a4ece4f44bd595ef245b,n/d,NaN


In [11]:
# 원본을 보니 별 내용이 없음.
origin_reviews = pd.read_csv(origin_reviews_path)
origin_reviews[origin_reviews['review_id'] == 'e9a917ef5451a4ece4f44bd595ef245b']

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
51119,e9a917ef5451a4ece4f44bd595ef245b,b4700fc91bb3fbc155bfc04ae821ba04,4,NaN,N/D,2017-02-23 00:00:00,2017-02-23 23:17:26


In [12]:
# 다른 order_id에 동일한 리뷰를 작성할 수도 있음
duplicated_rows = preprocessed_with_id_df[preprocessed_with_id_df.duplicated(keep=False)]
# duplicated_rows.sort_values('review_id').head(2)
origin_reviews[origin_reviews['review_id'] == "00130cbe1f9d422698c812ed8ded1919"]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
29841,00130cbe1f9d422698c812ed8ded1919,04a28263e085d399c97ae49e0b477efa,1,NaN,"O cartucho ""original HP"" 60XL não é reconhecido pela impressora, consequentemente não funcionou. Além de ter chegado com atraso de mais de 15 dias do previsto. Preciso que seja trocado.",2018-03-07 00:00:00,2018-03-20 18:08:23
46678,00130cbe1f9d422698c812ed8ded1919,dfcdfc43867d1c1381bfaf62d6b9c195,1,NaN,"O cartucho ""original HP"" 60XL não é reconhecido pela impressora, consequentemente não funcionou. Além de ter chegado com atraso de mais de 15 dias do previsto. Preciso que seja trocado.",2018-03-07 00:00:00,2018-03-20 18:08:23


In [13]:
# 중복 제거
preprocessed_with_id_df = preprocessed_with_id_df.drop_duplicates()
preprocessed_with_id_df.shape

(42273, 7)

In [14]:
print(len(origin_reviews))
origin_reviews = origin_reviews.drop(columns = ['review_comment_title', 'review_comment_message'])

os.makedirs(ARTIFACT_DIR, exist_ok=True)
result = []
for target, save_name in zip(['e2k', 'p2e'], ['kor', 'eng']):
    output_df = pd.merge(origin_reviews, preprocessed_with_id_df[['review_id', f'title_{target}', f'message_{target}']], on='review_id', how='left')

    save_path = os.path.join(ARTIFACT_DIR, f'{save_name}.csv')
    output_df.to_csv(save_path, index=False)

    result.append(output_df)

    print(len(output_df))

99224
99224
99224


# 감성분석 결과 추가

In [15]:
reviews_eng_df = output_df.copy()
reviews_eng_df['sentimental_title'] = reviews_eng_df['title_p2e']
reviews_eng_df['sentimental_message'] = reviews_eng_df['message_p2e']
reviews_eng_df.head()

,review_id,order_id,review_score,review_creation_date,review_answer_timestamp,title_p2e,message_p2e,sentimental_title,sentimental_message
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,2018-01-18 00:00:00,2018-01-18 21:46:59,NaN,NaN,NaN,NaN
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,2018-03-10 00:00:00,2018-03-11 03:05:13,NaN,NaN,NaN,NaN
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,2018-02-17 00:00:00,2018-02-18 14:36:24,NaN,NaN,NaN,NaN
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,2017-04-21 00:00:00,2017-04-21 22:02:06,NaN,I received it well before the stipulated deadline.,NaN,I received it well before the stipulated deadline.
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,2018-03-01 00:00:00,2018-03-02 10:26:53,NaN,congratulations lannister stores i loved buying online safe and practical congratulations to everyone happy easter,NaN,congratulations lannister stores i loved buying online safe and practical congratulations to everyone happy easter


In [16]:
p2e_df = merged_translation[['p2e']]
p2e_df.shape

(38103, 1)

In [17]:
sentimental_score_path = os.path.join(TRANSLATED_RESULT_DIR, 'senti_eng.csv')
sentimental_score_df = pd.read_csv(sentimental_score_path)
sentimental_score_df.head()

,negative,neutral,positive
0,0.999187,0.000605,0.000208
1,0.772191,0.226874,0.000935
2,0.998707,0.001103,0.000190
3,0.999120,0.000674,0.000206
4,0.998285,0.001533,0.000182


In [18]:
max_senti_df = pd.DataFrame(sentimental_score_df.idxmax(axis=1), columns=['sentimental'])
max_senti_df

,sentimental
0,negative
1,negative
2,negative
3,negative
4,negative
...,...
38098,neutral
38099,neutral
38100,neutral
38101,neutral


In [19]:
sentimetal_p2e_df = pd.concat([p2e_df, max_senti_df], axis=1)
sentimetal_p2e_df.head()

,p2e,sentimental
0,"“I did not receive the product, the product is listed as sent to the post office in Ribeirão Preto. The post office did not receive the product. I sent several messages to Targaryen and did not receive a response. This Targaryen is ruining her name”",negative
1,"I bought 2 backpacks from the lannister site and from different partners. targaryen from mg delivered well before the estimated deadline, while the other one from pacific that comes from sc still hasn’t been delivered. congratulations to targaryen",negative
2,"I bought the product trusting the lannister stores and not the targaryen one which seems to not have delivered the product in the right time for the post office, and I did not receive any clarification from the targaryen, but the product arrived, thank you",negative
3,"as soon as I made the request I noticed that I had ordered the wrong product and immediately asked for the order to be cancelled but I did not receive a response from targaryen, I have already made more than 10 requests for a refund but they do not respond",negative
4,in my purchase made on 09/20/17 the targaryen store did not have the merchandise and until now neither targaryen nor stark have resolved. the merchandise and shipping were paid for and this is what I think should be refunded.,negative


In [20]:
sentimetal_p2e_df.shape

(38103, 2)

In [21]:
sentimetal_p2e_df = sentimetal_p2e_df.drop_duplicates()
sentimetal_p2e_df.shape

(36026, 2)

In [22]:
duplicated_eng = sentimetal_p2e_df[sentimetal_p2e_df['p2e'].duplicated(keep=False)].sort_values('p2e')
duplicated_eng

,p2e,sentimental
35148,I would recommend it,positive
36160,I would recommend it,neutral


In [23]:
# 동일 문장에 대한 다른 스코어를 하드코딩 처리
# Issue: https://github.com/jmhwang-dev/e-commerce/issues/30
sentimetal_p2e_df.loc[36160, 'sentimental'] = 'positive'
duplicated_eng = sentimetal_p2e_df[sentimetal_p2e_df['p2e'].duplicated(keep=False)].sort_values('p2e')
duplicated_eng

,p2e,sentimental
35148,I would recommend it,positive
36160,I would recommend it,positive


In [24]:
sentimetal_p2e_df = sentimetal_p2e_df.drop_duplicates()
sentimetal_p2e_df.shape

(36025, 2)

In [25]:
senti_dict = dict(zip(sentimetal_p2e_df["p2e"], sentimetal_p2e_df['sentimental']))

In [26]:
reviews_eng_df['sentimental_title'] = reviews_eng_df["sentimental_title"].map(senti_dict)
reviews_eng_df['sentimental_message'] = reviews_eng_df["message_p2e"].map(senti_dict)
reviews_eng_df.head()

,review_id,order_id,review_score,review_creation_date,review_answer_timestamp,title_p2e,message_p2e,sentimental_title,sentimental_message
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,2018-01-18 00:00:00,2018-01-18 21:46:59,NaN,NaN,neutral,neutral
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,2018-03-10 00:00:00,2018-03-11 03:05:13,NaN,NaN,neutral,neutral
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,2018-02-17 00:00:00,2018-02-18 14:36:24,NaN,NaN,neutral,neutral
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,2017-04-21 00:00:00,2017-04-21 22:02:06,NaN,I received it well before the stipulated deadline.,neutral,negative
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,2018-03-01 00:00:00,2018-03-02 10:26:53,NaN,congratulations lannister stores i loved buying online safe and practical congratulations to everyone happy easter,neutral,positive


In [27]:
# title과 comment가 감성이 다른 경우가 있음
reviews_eng_df.groupby(['sentimental_title', "sentimental_message"])['review_id'].count()

sentimental_title  sentimental_message
negative           negative                1556
                   neutral                  440
                   positive                  42
neutral            negative               10220
                   neutral                64300
                   positive               15689
positive           negative                 376
                   neutral                 2457
                   positive                4144
Name: review_id, dtype: int64

In [28]:
both_na_mask = reviews_eng_df['sentimental_title'].isna() & reviews_eng_df['sentimental_message'].isna()
reviews_eng_df[both_na_mask]

,review_id,order_id,review_score,review_creation_date,review_answer_timestamp,title_p2e,message_p2e,sentimental_title,sentimental_message


# 최종 감성분석 스코어 판단 기준
> 감성분석 추론 결과 의심사항: ex) 예정된 배송일 전에 받았다는 내용에 대해서 `negative`로 판단함. 추후 확인 필요

1. 리뷰가 없으면 스코어로 감성을 판단한다.

2. 감성분석 판단의 우선순위: 리뷰(제목+내용) > 스코어
    - 제목과 내용이 상반되는 내용이라면, 내용 > 제목

In [29]:
score_mapping = {
    1: 'negative',
    2: 'negative',
    3: 'neutral',
    4: 'positive',
    5: 'positive'
}

reviews_eng_df['sentiment_from_score'] = reviews_eng_df['review_score'].map(score_mapping)
reviews_eng_df['sentiment_from_score'].value_counts(dropna=False)

sentiment_from_score
positive    76470
negative    14575
neutral      8179
Name: count, dtype: int64

In [30]:
def get_content_sentiment(row):
    title = row['title_p2e']
    message = row['message_p2e']
    sentimental_message = row['sentimental_message']
    
    if pd.isna(title) and pd.isna(message):
        return np.nan
    
    return sentimental_message

reviews_eng_df['sentimental_from_content'] = reviews_eng_df.apply(get_content_sentiment, axis=1)
reviews_eng_df['sentimental_from_content'].value_counts(dropna=False)

sentimental_from_content
NaN         56636
positive    19875
negative    12152
neutral     10561
Name: count, dtype: int64

In [31]:
def get_overall_sentiment(row):
    sentimental_from_content = row['sentimental_from_content']
    sentiment_from_score = row['sentiment_from_score']
    
    if pd.isna(sentimental_from_content):
        return sentiment_from_score
    
    return sentimental_from_content

reviews_eng_df['sentimental_overall'] = reviews_eng_df.apply(get_overall_sentiment, axis=1)
reviews_eng_df['sentimental_overall'].value_counts(dropna=False)

sentimental_overall
positive    68383
negative    15737
neutral     15104
Name: count, dtype: int64

In [32]:
# reviews_eng_df.drop(columns=["title_p2e", 'message_p2e'], inplace=True)

In [33]:
reviews_eng_df.to_csv("../artifact/reviews/eng_senti.csv", index=False)